In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, Input
from tensorflow.keras import layers
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, concatenate
from pandas import concat
from pandas import read_csv
from helper import series_to_supervised, stage_series_to_supervised

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2, 7"

In [30]:
dataset = pd.read_csv('../../../data/TFT_energy.csv', index_col=0)
dataset.fillna(0, inplace=True)
data = dataset
data

,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,pressure_Bilbao,...,wind_deg_Bilbao,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,64.92,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,1035.0,...,229.0,0.0,1009.0,21.0,7.0,1.0,0.0,5890.0,1.0,920.0
2015-01-01 01:00:00+00:00,64.48,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,1036.0,...,224.0,0.0,973.0,27.0,7.0,0.0,1.0,5461.0,1.0,1164.0
2015-01-01 02:00:00+00:00,59.32,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,1035.0,...,225.0,0.0,949.0,27.0,7.0,0.0,1.0,5238.0,1.0,1503.0
2015-01-01 03:00:00+00:00,56.04,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,1035.0,...,221.0,0.0,953.0,27.0,7.0,0.0,1.0,4935.0,1.0,1826.0
2015-01-01 04:00:00+00:00,53.63,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,1035.0,...,224.0,0.0,952.0,57.0,5.0,2.0,1.0,4618.0,0.0,2109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,1033.0,...,57.0,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0
2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,1034.0,...,0.0,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0
2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,1034.0,...,140.0,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0


In [31]:
print(dataset.columns)

Index(['price', 'price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump'],
      dtype='object')


In [32]:
# specify the number of lag hours
n_hours = 24*3
K = 24

### Target time series (4)
- 'WS_S1',  'TWS_S25A', 'TWS_S25B', 'TWS_S26',    
        
               
### Prior known covariates (9)
- 'WS_S4', 'GATE_S25A', 'GATE_S25B', 'GATE_S25B2', 'PUMP_S25B',  'GATE_S26_1', 'GATE_S26_2', 'PUMP_S26', 'MEAN_RAIN'


### Prior unknown covariates (6)
- 'FLOW_S25A', 'HWS_S25A', 'FLOW_S25B', 'HWS_S25B', 'FLOW_S26', 'HWS_S26'

#### Target time series: water stage (4)

In [33]:
# Pre-processing
stages = dataset[['price']]
print("stages.shape:", stages.shape)


stages_supervised = series_to_supervised(stages, n_hours, K)
print("stages_supervised.shape:", stages_supervised.shape)

stages.shape: (35063, 1)
stages_supervised.shape: (34968, 96)


In [34]:
stages_supervised

,var1(t-72),var1(t-71),var1(t-70),var1(t-69),var1(t-68),var1(t-67),var1(t-66),var1(t-65),var1(t-64),var1(t-63),...,var1(t+14),var1(t+15),var1(t+16),var1(t+17),var1(t+18),var1(t+19),var1(t+20),var1(t+21),var1(t+22),var1(t+23)
time,,,,,,,,,,,,,,,,,,,,,
2015-01-04 00:00:00+00:00,64.92,64.48,59.32,56.04,53.63,51.73,51.43,48.98,54.20,58.94,...,52.07,52.15,61.12,71.98,74.64,75.38,76.49,73.83,69.51,70.77
2015-01-04 01:00:00+00:00,64.48,59.32,56.04,53.63,51.73,51.43,48.98,54.20,58.94,59.86,...,52.15,61.12,71.98,74.64,75.38,76.49,73.83,69.51,70.77,64.89
2015-01-04 02:00:00+00:00,59.32,56.04,53.63,51.73,51.43,48.98,54.20,58.94,59.86,60.12,...,61.12,71.98,74.64,75.38,76.49,73.83,69.51,70.77,64.89,60.91
2015-01-04 03:00:00+00:00,56.04,53.63,51.73,51.43,48.98,54.20,58.94,59.86,60.12,62.05,...,71.98,74.64,75.38,76.49,73.83,69.51,70.77,64.89,60.91,59.68
2015-01-04 04:00:00+00:00,53.63,51.73,51.43,48.98,54.20,58.94,59.86,60.12,62.05,62.06,...,74.64,75.38,76.49,73.83,69.51,70.77,64.89,60.91,59.68,58.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-30 19:00:00+00:00,78.99,77.65,73.28,73.48,70.93,68.61,64.19,62.53,59.78,59.82,...,72.12,71.81,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02
2018-12-30 20:00:00+00:00,77.65,73.28,73.48,70.93,68.61,64.19,62.53,59.78,59.82,64.27,...,71.81,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16
2018-12-30 21:00:00+00:00,73.28,73.48,70.93,68.61,64.19,62.53,59.78,59.82,64.27,66.81,...,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16,74.30


#### Prior unknown covariates (6)

In [35]:
# prior_unknown = dataset[['FLOW_S25A', 'HWS_S25A', 'FLOW_S25B', 'HWS_S25B', 'FLOW_S26', 'HWS_S26' ]]
# print("prior_unknown.shape:", prior_unknown.shape)

# prior_unknown_supervised = series_to_supervised(prior_unknown, n_hours, K)
# print("prior_unknown_supervised.shape:", prior_unknown_supervised.shape)

In [36]:
# prior_unknown_supervised

#### Prior known covariates (9)

In [37]:
prior_known = dataset[['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump']]
print("prior_known.shape:", prior_known.shape)

prior_known_supervised = series_to_supervised(prior_known, n_hours, K)
print("prior_known_supervised.shape:", prior_known_supervised.shape)

prior_known.shape: (35063, 25)
prior_known_supervised.shape: (34968, 2400)


In [38]:
# prior_known_supervised

### N_out, N_in, K

In [39]:
past_cov = dataset[['price_dayahead', 'gen_coal', 'gen_gas', 'load_actual',
       'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona',
       'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia',
       'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville',
       'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river',
       'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia',
       'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump']]

past_cov_supervised = series_to_supervised(past_cov, n_hours, K)
print("past_cov_supervised.shape:", past_cov_supervised.shape)

past_cov_supervised.shape: (34968, 2400)


In [40]:
# past_cov_supervised  # 6 + 9 = 15 features

In [41]:
past_ws_supervised = stages_supervised.iloc[:, :n_hours*stages.shape[1]]
past_ws_supervised.reset_index(drop=True, inplace=True)
past_ws_supervised.columns = ['past_ws_supervised_' + i for i in list(past_ws_supervised.columns)]

In [42]:
# past_ws_supervised  # 4

In [43]:
columns = []
for i in range(n_hours):
    columns = columns + past_cov_supervised.columns[i*past_cov.shape[1]:(i+1)*past_cov.shape[1]].tolist()
    columns = columns + past_ws_supervised.columns[i*stages.shape[1]:(i+1)*stages.shape[1]].tolist()
# columns

In [44]:
past_cov_supervised.reset_index(drop=True, inplace=True)
past_ws_supervised.reset_index(drop=True, inplace=True)

past_cov_ws_supervised = pd.concat([past_cov_supervised, past_ws_supervised], axis=1)
past_cov_ws_supervised = past_cov_ws_supervised[columns]


In [45]:
# past_cov_ws_supervised

In [46]:
shift_prior_known_supervised = prior_known_supervised.iloc[:, K*prior_known.shape[1]:]  # shift 2 days for prior known covariates
shift_prior_known_supervised.reset_index(drop=True, inplace=True)
shift_prior_known_supervised.columns = ['shift_prior_known_supervised_' + i for i in list(shift_prior_known_supervised.columns)]
shift_prior_known_supervised

,shift_prior_known_supervised_var1(t-48),shift_prior_known_supervised_var2(t-48),shift_prior_known_supervised_var3(t-48),shift_prior_known_supervised_var4(t-48),shift_prior_known_supervised_var5(t-48),shift_prior_known_supervised_var6(t-48),shift_prior_known_supervised_var7(t-48),shift_prior_known_supervised_var8(t-48),shift_prior_known_supervised_var9(t-48),shift_prior_known_supervised_var10(t-48),...,shift_prior_known_supervised_var16(t+23),shift_prior_known_supervised_var17(t+23),shift_prior_known_supervised_var18(t+23),shift_prior_known_supervised_var19(t+23),shift_prior_known_supervised_var20(t+23),shift_prior_known_supervised_var21(t+23),shift_prior_known_supervised_var22(t+23),shift_prior_known_supervised_var23(t+23),shift_prior_known_supervised_var24(t+23),shift_prior_known_supervised_var25(t+23)
0,7.00,1190.0,2902.0,24935.0,0.0,192.0,61.0,1038.0,1036.0,167.0,...,156.0,21.0,1182.0,132.0,2.0,1.0,0.0,7351.0,1.0,650.0
1,5.00,1023.0,2772.0,23214.0,0.0,189.0,60.0,1037.0,1036.0,167.0,...,163.0,12.0,1099.0,132.0,1.0,1.0,0.0,7403.0,1.0,938.0
2,4.00,1016.0,2936.0,22540.0,0.0,188.0,59.0,1037.0,1036.0,181.0,...,160.0,13.0,1076.0,108.0,2.0,2.0,0.0,7887.0,1.0,1021.0
3,4.00,1103.0,2893.0,22096.0,0.0,189.0,61.0,1037.0,1037.0,181.0,...,165.0,6.0,982.0,108.0,1.0,2.0,0.0,8348.0,1.0,2067.0
4,4.80,1120.0,2889.0,22066.0,0.0,190.0,59.0,1038.0,1037.0,171.0,...,162.0,20.0,982.0,108.0,1.0,2.0,0.0,8348.0,1.0,2067.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34963,66.00,2931.0,9149.0,32642.0,718.0,294.0,103.0,1028.0,1033.0,314.0,...,57.0,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0
34964,62.09,2889.0,8024.0,32155.0,675.0,294.0,103.0,1028.0,1033.0,315.0,...,0.0,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0
34965,61.77,2871.0,6871.0,30428.0,569.0,291.0,104.0,1029.0,1033.0,314.0,...,140.0,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0
34966,58.43,2619.0,5822.0,28015.0,310.0,288.0,103.0,1029.0,1033.0,313.0,...,120.0,0.0,1128.0,60.0,5.0,2.0,1.0,3586.0,2.0,108.0


In [47]:
shift_prior_known_past_cov_ws_supervised = pd.concat([past_cov_ws_supervised, shift_prior_known_supervised], 
                                                     axis=1)


In [48]:
# shift_prior_known_past_cov_ws_supervised

In [49]:
future_ws_supervised = stages_supervised.iloc[:, n_hours*stages.shape[1]:]
future_ws_supervised.reset_index(drop=True, inplace=True)
future_ws_supervised

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),...,var1(t+14),var1(t+15),var1(t+16),var1(t+17),var1(t+18),var1(t+19),var1(t+20),var1(t+21),var1(t+22),var1(t+23)
0,50.54,48.68,48.02,47.06,46.79,47.63,47.44,50.84,54.84,59.95,...,52.07,52.15,61.12,71.98,74.64,75.38,76.49,73.83,69.51,70.77
1,48.68,48.02,47.06,46.79,47.63,47.44,50.84,54.84,59.95,59.35,...,52.15,61.12,71.98,74.64,75.38,76.49,73.83,69.51,70.77,64.89
2,48.02,47.06,46.79,47.63,47.44,50.84,54.84,59.95,59.35,57.03,...,61.12,71.98,74.64,75.38,76.49,73.83,69.51,70.77,64.89,60.91
3,47.06,46.79,47.63,47.44,50.84,54.84,59.95,59.35,57.03,59.22,...,71.98,74.64,75.38,76.49,73.83,69.51,70.77,64.89,60.91,59.68
4,46.79,47.63,47.44,50.84,54.84,59.95,59.35,57.03,59.22,58.56,...,74.64,75.38,76.49,73.83,69.51,70.77,64.89,60.91,59.68,58.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34963,73.35,73.64,71.92,69.49,68.40,60.32,57.73,57.66,57.04,57.48,...,72.12,71.81,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02
34964,73.64,71.92,69.49,68.40,60.32,57.73,57.66,57.04,57.48,61.13,...,71.81,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16
34965,71.92,69.49,68.40,60.32,57.73,57.66,57.04,57.48,61.13,60.70,...,72.42,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16,74.30
34966,69.49,68.40,60.32,57.73,57.66,57.04,57.48,61.13,60.70,66.64,...,71.80,71.95,70.85,71.36,75.13,77.61,77.02,76.16,74.30,69.89


### Concatenation

In [50]:
all_data = concat([shift_prior_known_past_cov_ws_supervised, future_ws_supervised], axis=1)
# print("all_data", all_data)
print("all_data.shape:", all_data.shape)

all_data.shape: (34968, 3696)


### Train & Test set

In [51]:
all_data = all_data.values
n_train_hours = int(len(all_data)*0.8)
print("n_train_hours:", n_train_hours)


train = all_data[:n_train_hours, :]
test = all_data[n_train_hours:, :]

n_train_hours: 27974


In [52]:
n_obs = n_hours * (prior_known.shape[1]+data.shape[1])
n_obs

3672

### Normalization

In [53]:
# split into input and outputs
all_features = prior_known.shape[1] + data.shape[1]
n_obs = n_hours * all_features
train_X, train_y = train[:, :n_obs], train[:, n_obs:]
test_X, test_y = test[:, :n_obs], test[:, n_obs:]
print("train_X.shape, train_y.shape, test_X.shape, test_y.shape", train_X.shape, train_y.shape, test_X.shape, test_y.shape)


# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
train_X = scaler.fit_transform(train_X)
train_y = scaler.fit_transform(train_y)
test_X = scaler.fit_transform(test_X)
test_y = scaler.fit_transform(test_y)

train_X.shape, train_y.shape, test_X.shape, test_y.shape (27974, 3672) (27974, 24) (6994, 3672) (6994, 24)


In [54]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, all_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, all_features))
print("train_X.shape, train_y.shape, test_X.shape, test_y.shape: \n", train_X.shape, train_y.shape, test_X.shape, test_y.shape)

train_X.shape, train_y.shape, test_X.shape, test_y.shape: 
 (27974, 72, 51) (27974, 24) (6994, 72, 51) (6994, 24)


### Model

In [64]:
model_input = Input(shape=(train_X.shape[1], train_X.shape[2]))
x = layers.SimpleRNN(128, 
                     activation='relu', 
                     kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                     recurrent_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                     return_sequences=True)(model_input)

x_merge1 = concatenate([model_input, x])
x = layers.SimpleRNN(64, 
                     activation='relu', 
                     kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                     recurrent_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                     return_sequences=True)(x_merge1)

x_merge2 = concatenate([model_input, x])
x = layers.SimpleRNN(32, 
                     activation='relu', 
                     kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                     recurrent_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                     return_sequences=True)(x_merge2)

x_merge3 = concatenate([model_input, x])
x = layers.SimpleRNN(16, 
                     activation='relu', 
                     kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                     recurrent_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                     return_sequences=True)(x_merge3)

merge = concatenate([model_input, x])

x = layers.Flatten()(merge)
x = Dense(256, activation='relu', kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5))(x)
x = Dense(128, activation='relu', kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5))(x)
x = Dense(64, activation='relu', kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5))(x)

output = layers.Dense(train_y.shape[1])(x)

model_rnn = Model(model_input, output)
model_rnn.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 72, 51)]     0                                            
__________________________________________________________________________________________________
simple_rnn_17 (SimpleRNN)       (None, 72, 128)      23040       input_6[0][0]                    
__________________________________________________________________________________________________
concatenate_16 (Concatenate)    (None, 72, 179)      0           input_6[0][0]                    
                                                                 simple_rnn_17[0][0]              
__________________________________________________________________________________________________
simple_rnn_18 (SimpleRNN)       (None, 72, 64)       15616       concatenate_16[0][0]       

In [65]:
lr = 0.00001
EPOCHS = 6000

# initial_learning_rate * decay_rate ^ (step / decay_steps)
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=5e-4, 
                                                          decay_steps=10000,
                                                          decay_rate=0.99)

model_rnn.compile(
              optimizer=Adam(learning_rate=lr_schedule), 
#               optimizer='adam',
              loss='mse',
              metrics=['mae'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=1000)
mc = ModelCheckpoint('saved_model/rnn_shift_anyday_24h_4layer_sc.h5', monitor='val_mae', mode='min', verbose=2, save_best_only=True)


history = model_rnn.fit(train_X, train_y,
                    batch_size=512,
                    epochs=EPOCHS,
                    validation_data=(test_X, test_y),
                    verbose=2,
                    shuffle=True,
                       callbacks=[es, mc])

plt.rcParams["figure.figsize"] = (8, 6)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.legend(fontsize=14)
plt.title("Training loss vs Testing loss", fontsize=18)
# plt.savefig('graph/rnn_loss.png', dpi=300)
plt.show()

Epoch 1/6000

Epoch 00001: val_mae improved from inf to 0.19827, saving model to saved_model/rnn_shift_anyday_24h_4layer_sc.h5
55/55 - 10s - loss: 0.3085 - mae: 0.1482 - val_loss: 0.2815 - val_mae: 0.1983
Epoch 2/6000

Epoch 00002: val_mae improved from 0.19827 to 0.15327, saving model to saved_model/rnn_shift_anyday_24h_4layer_sc.h5
55/55 - 9s - loss: 0.2287 - mae: 0.0813 - val_loss: 0.2378 - val_mae: 0.1533
Epoch 3/6000

Epoch 00003: val_mae improved from 0.15327 to 0.11090, saving model to saved_model/rnn_shift_anyday_24h_4layer_sc.h5
55/55 - 9s - loss: 0.2051 - mae: 0.0703 - val_loss: 0.2057 - val_mae: 0.1109
Epoch 4/6000

Epoch 00004: val_mae improved from 0.11090 to 0.09190, saving model to saved_model/rnn_shift_anyday_24h_4layer_sc.h5
55/55 - 9s - loss: 0.1868 - mae: 0.0640 - val_loss: 0.1850 - val_mae: 0.0919
Epoch 5/6000

Epoch 00005: val_mae improved from 0.09190 to 0.08311, saving model to saved_model/rnn_shift_anyday_24h_4layer_sc.h5
55/55 - 9s - loss: 0.1717 - mae: 0.0605 

Epoch 52/6000

Epoch 00052: val_mae improved from 0.04958 to 0.04906, saving model to saved_model/rnn_shift_anyday_24h_4layer_sc.h5
55/55 - 9s - loss: 0.0146 - mae: 0.0429 - val_loss: 0.0153 - val_mae: 0.0491
Epoch 53/6000

Epoch 00053: val_mae did not improve from 0.04906
55/55 - 9s - loss: 0.0143 - mae: 0.0431 - val_loss: 0.0156 - val_mae: 0.0543
Epoch 54/6000

Epoch 00054: val_mae did not improve from 0.04906
55/55 - 9s - loss: 0.0141 - mae: 0.0442 - val_loss: 0.0155 - val_mae: 0.0546
Epoch 55/6000

Epoch 00055: val_mae did not improve from 0.04906
55/55 - 9s - loss: 0.0137 - mae: 0.0437 - val_loss: 0.0147 - val_mae: 0.0515
Epoch 56/6000

Epoch 00056: val_mae did not improve from 0.04906
55/55 - 9s - loss: 0.0134 - mae: 0.0433 - val_loss: 0.0142 - val_mae: 0.0502
Epoch 57/6000

Epoch 00057: val_mae did not improve from 0.04906
55/55 - 9s - loss: 0.0131 - mae: 0.0433 - val_loss: 0.0140 - val_mae: 0.0512
Epoch 58/6000

Epoch 00058: val_mae did not improve from 0.04906
55/55 - 9s - los

Epoch 107/6000

Epoch 00107: val_mae did not improve from 0.04572
55/55 - 9s - loss: 0.0065 - mae: 0.0412 - val_loss: 0.0072 - val_mae: 0.0469
Epoch 108/6000

Epoch 00108: val_mae did not improve from 0.04572
55/55 - 9s - loss: 0.0063 - mae: 0.0406 - val_loss: 0.0089 - val_mae: 0.0601
Epoch 109/6000

Epoch 00109: val_mae did not improve from 0.04572
55/55 - 9s - loss: 0.0063 - mae: 0.0407 - val_loss: 0.0083 - val_mae: 0.0558
Epoch 110/6000

Epoch 00110: val_mae improved from 0.04572 to 0.04520, saving model to saved_model/rnn_shift_anyday_24h_4layer_sc.h5
55/55 - 9s - loss: 0.0062 - mae: 0.0406 - val_loss: 0.0069 - val_mae: 0.0452
Epoch 111/6000

Epoch 00111: val_mae did not improve from 0.04520
55/55 - 9s - loss: 0.0064 - mae: 0.0420 - val_loss: 0.0083 - val_mae: 0.0568
Epoch 112/6000

Epoch 00112: val_mae did not improve from 0.04520
55/55 - 9s - loss: 0.0062 - mae: 0.0413 - val_loss: 0.0070 - val_mae: 0.0470
Epoch 113/6000

Epoch 00113: val_mae did not improve from 0.04520
55/55 - 9

Epoch 164/6000

Epoch 00164: val_mae did not improve from 0.04430
55/55 - 9s - loss: 0.0051 - mae: 0.0401 - val_loss: 0.0061 - val_mae: 0.0478
Epoch 165/6000

Epoch 00165: val_mae did not improve from 0.04430
55/55 - 9s - loss: 0.0051 - mae: 0.0401 - val_loss: 0.0061 - val_mae: 0.0479
Epoch 166/6000

Epoch 00166: val_mae did not improve from 0.04430
55/55 - 9s - loss: 0.0051 - mae: 0.0404 - val_loss: 0.0070 - val_mae: 0.0552
Epoch 167/6000

Epoch 00167: val_mae did not improve from 0.04430
55/55 - 9s - loss: 0.0051 - mae: 0.0399 - val_loss: 0.0057 - val_mae: 0.0449
Epoch 168/6000

Epoch 00168: val_mae did not improve from 0.04430
55/55 - 9s - loss: 0.0050 - mae: 0.0396 - val_loss: 0.0057 - val_mae: 0.0443
Epoch 169/6000

Epoch 00169: val_mae did not improve from 0.04430
55/55 - 9s - loss: 0.0050 - mae: 0.0396 - val_loss: 0.0063 - val_mae: 0.0490
Epoch 170/6000

Epoch 00170: val_mae did not improve from 0.04430
55/55 - 9s - loss: 0.0051 - mae: 0.0408 - val_loss: 0.0067 - val_mae: 0.0525

Epoch 222/6000

Epoch 00222: val_mae did not improve from 0.04430
55/55 - 9s - loss: 0.0047 - mae: 0.0390 - val_loss: 0.0059 - val_mae: 0.0480
Epoch 223/6000

Epoch 00223: val_mae did not improve from 0.04430
55/55 - 9s - loss: 0.0047 - mae: 0.0393 - val_loss: 0.0060 - val_mae: 0.0498
Epoch 224/6000

Epoch 00224: val_mae did not improve from 0.04430
55/55 - 9s - loss: 0.0047 - mae: 0.0388 - val_loss: 0.0063 - val_mae: 0.0513
Epoch 225/6000

Epoch 00225: val_mae did not improve from 0.04430
55/55 - 9s - loss: 0.0048 - mae: 0.0402 - val_loss: 0.0056 - val_mae: 0.0457
Epoch 226/6000

Epoch 00226: val_mae did not improve from 0.04430
55/55 - 9s - loss: 0.0048 - mae: 0.0402 - val_loss: 0.0058 - val_mae: 0.0470
Epoch 227/6000

Epoch 00227: val_mae did not improve from 0.04430
55/55 - 9s - loss: 0.0046 - mae: 0.0385 - val_loss: 0.0062 - val_mae: 0.0507
Epoch 228/6000

Epoch 00228: val_mae did not improve from 0.04430
55/55 - 9s - loss: 0.0046 - mae: 0.0387 - val_loss: 0.0076 - val_mae: 0.0597

...

Epoch 3986/6000

Epoch 03986: val_mae did not improve from 0.04163
55/55 - 9s - loss: 0.0036 - mae: 0.0368 - val_loss: 0.0047 - val_mae: 0.0448
Epoch 3987/6000

Epoch 03987: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0036 - mae: 0.0360 - val_loss: 0.0049 - val_mae: 0.0465
Epoch 3988/6000

Epoch 03988: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0039 - mae: 0.0390 - val_loss: 0.0059 - val_mae: 0.0537
Epoch 3989/6000

Epoch 03989: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0038 - mae: 0.0377 - val_loss: 0.0047 - val_mae: 0.0457
Epoch 3990/6000

Epoch 03990: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0036 - mae: 0.0360 - val_loss: 0.0056 - val_mae: 0.0523
Epoch 3991/6000

Epoch 03991: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0036 - mae: 0.0362 - val_loss: 0.0047 - val_mae: 0.0446
Epoch 3992/6000

Epoch 03992: val_mae did not improve from 0.04163
55/55 - 9s - loss: 0.0038 - mae: 0.0379 - val_loss: 0.0049 - val

Epoch 4043/6000

Epoch 04043: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0036 - mae: 0.0363 - val_loss: 0.0060 - val_mae: 0.0548
Epoch 4044/6000

Epoch 04044: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0039 - mae: 0.0384 - val_loss: 0.0046 - val_mae: 0.0437
Epoch 4045/6000

Epoch 04045: val_mae did not improve from 0.04163
55/55 - 9s - loss: 0.0037 - mae: 0.0364 - val_loss: 0.0047 - val_mae: 0.0451
Epoch 4046/6000

Epoch 04046: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0036 - mae: 0.0362 - val_loss: 0.0053 - val_mae: 0.0497
Epoch 4047/6000

Epoch 04047: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0036 - mae: 0.0362 - val_loss: 0.0046 - val_mae: 0.0441
Epoch 4048/6000

Epoch 04048: val_mae did not improve from 0.04163
55/55 - 9s - loss: 0.0036 - mae: 0.0365 - val_loss: 0.0056 - val_mae: 0.0514
Epoch 4049/6000

Epoch 04049: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0037 - mae: 0.0373 - val_loss: 0.0067 - val

Epoch 4100/6000

Epoch 04100: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0037 - mae: 0.0373 - val_loss: 0.0053 - val_mae: 0.0498
Epoch 4101/6000

Epoch 04101: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0036 - mae: 0.0359 - val_loss: 0.0057 - val_mae: 0.0526
Epoch 4102/6000

Epoch 04102: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0036 - mae: 0.0361 - val_loss: 0.0049 - val_mae: 0.0472
Epoch 4103/6000

Epoch 04103: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0036 - mae: 0.0360 - val_loss: 0.0047 - val_mae: 0.0453
Epoch 4104/6000

Epoch 04104: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0036 - mae: 0.0360 - val_loss: 0.0051 - val_mae: 0.0488
Epoch 4105/6000

Epoch 04105: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0036 - mae: 0.0364 - val_loss: 0.0045 - val_mae: 0.0427
Epoch 4106/6000

Epoch 04106: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0035 - mae: 0.0357 - val_loss: 0.0045 - v

Epoch 4157/6000

Epoch 04157: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0036 - mae: 0.0361 - val_loss: 0.0048 - val_mae: 0.0457
Epoch 4158/6000

Epoch 04158: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0036 - mae: 0.0361 - val_loss: 0.0051 - val_mae: 0.0485
Epoch 4159/6000

Epoch 04159: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0037 - mae: 0.0376 - val_loss: 0.0050 - val_mae: 0.0474
Epoch 4160/6000

Epoch 04160: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0036 - mae: 0.0359 - val_loss: 0.0046 - val_mae: 0.0447
Epoch 4161/6000

Epoch 04161: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0036 - mae: 0.0364 - val_loss: 0.0048 - val_mae: 0.0462
Epoch 4162/6000

Epoch 04162: val_mae did not improve from 0.04163
55/55 - 10s - loss: 0.0036 - mae: 0.0362 - val_loss: 0.0053 - val_mae: 0.0504
Epoch 4163/6000

Epoch 04163: val_mae did not improve from 0.04163
55/55 - 11s - loss: 0.0038 - mae: 0.0381 - val_loss: 0.0056 - v

KeyboardInterrupt: 

### Performance

In [66]:
from tensorflow.keras.models import load_model
saved_model = load_model('saved_model/rnn_shift_anyday_24h_4layer_sc.h5')

In [67]:
yhat = saved_model.predict(test_X)
inv_yhat = scaler.inverse_transform(yhat)
inv_y = scaler.inverse_transform(test_y)

inv_yhat = pd.DataFrame(inv_yhat)
inv_y = pd.DataFrame(inv_y)
print("inv_y.shape, inv_yhat.shape", inv_y.shape, inv_yhat.shape)

error = inv_yhat - inv_y
print(error.shape)

inv_y.shape, inv_yhat.shape (6994, 24) (6994, 24)
(6994, 24)


In [68]:
print('MAE = {}'.format(float("{:.3f}".format(mae(inv_yhat.iloc[:, :], inv_y.iloc[:, :])))))
print('RMSE = {}'.format(float("{:.3f}".format(sqrt(mse(inv_yhat.iloc[:, :], inv_y.iloc[:, :]))))))

MAE = 3.383
RMSE = 4.545
